In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Set the paths for training and test data
# train_data_dir = '/content/drive/MyDrive/data/train'
# val_data_dir = '/content/drive/MyDrive/data/test'

# Define the batch size and number of epochs for training
batch_size = 16
epochs = 30
num_classes = 5
# Define the input shape of the images (300 was choosen so that we can be able to use EfficientNetB2 model later on)
input_shape = (128, 128, 3)

# train test directory
train_data_dir = 'Data/sample/train'
val_data_dir = 'Data/sample/val'
# Create data generators for training and test data
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical')

# Define CNN model
model = Sequential()
model.add(Conv2D(128, (3, 3), activation="relu", input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3),activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(Dropout(0.5))

model.add(Conv2D(32, (3, 3),activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(64))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['acc'])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

# Define the early stopping and model checkpoint callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras import regularizers
# Create an instance of the EfficientNetB2 model
base_model = EfficientNetV2S(include_top=False, weights="imagenet", input_shape=input_shape, pooling = 'max')

# Added a normalzation
x = base_model.output
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)

# Added a fully connected layer with 256 hidden units, regularizers and ReLU activation
x = Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)

# Added a dropout layer to prevent overfitting
x = Dropout(0.45, seed=123)(x)

# The final softmax layer with 8 output units
output = Dense(num_classes, activation='softmax')(x)

# created the model with the base model and the fully connected layers
model_EfficientNetV2 = Model(inputs=base_model.input, outputs=output)

# compile the model with the Adam optimizer
model_EfficientNetV2.compile(optimizer=Adamax(learning_rate=.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the train and validation generators
history = model_EfficientNetV2.fit(train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=val_generator.samples // batch_size,
        callbacks=[early_stopping, model_checkpoint]
        )

In [ ]:

# Train the model
history= model.fit(train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=val_generator.samples // batch_size,
        callbacks=[early_stopping, model_checkpoint]
        )

In [ ]:
#plotting the accuracy and loss of the model
import matplotlib.pyplot as plt
def plot_accuracy_loss(history):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.show()


In [ ]:
# # Save the model
# model.save('model.h5')